### Summary
This script prepares data related to food justice locations, including forward geocoding of addresses, for use with ArcGIS Pro. 
- Loads the file provided by the Client (Rainier Beach Action Coalition) that contains infomation about food locations
- Performs data cleaning and ETL steps to simplify forward geocoding and mapping
- Calls LocationIq API to obtain lat/long 
- Transforms output into AcrGIS Pro friendly format and writes out to Excel file

In [1]:
# Import libraries 

import pandas as pd
import requests
import time


##### Define reference objects 
- Source data path and ETL items
- Common LocationIq API parameters 

In [2]:
# File path and dtypes for client supplied data
# This file contains the FJ categories assigned by the client

rBeachFJ_path = r'C:\\Users\\' # Path to source data


rBeachFJ_dtypes = {'id':'object', 'zip':'object', 'Food Swamp':'object', 'Food Desert': 'object',
                   'Biopic Owned':'object', 'Free Food':'object', 'Farms':'object',
                   'Land Ownership':'object', 'Commercial Kitchens':'object'}


# List of map category column names 
mapCatCols = ['Food_Swamp', 'Food_Desert','BIPOC_Owned', 'Free_Food', 'Farms', 'Land_Ownership','Commercial_Kitchens']


# List of updated column names
rBeachFJ_newCols = ['LocID', 'FoodBusiness_YN', 'Business_Name', 'Business_Address', 'State', 'Zip', 'Website', 'Category', 'Food_Swamp', 'Food_Desert',
               'BIPOC_Owned', 'Free_Food', 'Farms', 'Land_Ownership','Commercial_Kitchens']

In [3]:
#View box, provide limits of returned locations
maxLat = '47.59209'
minLat = '47.48523'
maxLong = '-122.217'
minLong = '-122.34053'

# Create viewbox string, to simplify call to LocationIq API
viewboxString = maxLong + ',' +  maxLat + ',' +  minLong + ',' +  minLat

# Prepare dict with non-address parameters, for use with LocationIq API
locIqParams = {'key': 'pk.c9ffece8c3c21a0c517497d54fd77a44',
               'format' : 'json',
               'limit' : '1',
               'viewbox' : viewboxString,
               'bounded':'1'
              }


# Define dictionary to simplify picking columns for use with LocIQ API
# keys = LocIQ API parameters, values = column names 

splitAddy_keyMap = {'street' : 'StreetAddress',
                    'city' : 'City',
                    'state' : 'State',
                    'postalcode' : 'Zip'
                       }

##### Define functions
Several fucntions are used to simplify tasks. 
- Clean addresses
- Find street address, to standarize address format 
- Split addresses into API compliant dictionary 
- Get lat/long, using LocationIq API
- Remove NaN rows

In [4]:
# Clean addresses of pre-identified errors, except when error, return origional address

def clean_address(addy):
    
    addyErr = False
    
    cleanAddy = addy
    
    # Clean addresses, except when error, return origional address
    
    try: 
        cleanAddy = cleanAddy.replace('MLK', 'Martin Luther King')
        cleanAddy = cleanAddy.replace('Mlk', 'Martin Luther King Jr Way S')
        cleanAddy = cleanAddy.replace('\n', ', ')
        cleanAddy = cleanAddy.replace('SSeattle', 'S, Seattle')
        cleanAddy = cleanAddy.replace('StSeattle', 'St, Seattle')
        cleanAddy = cleanAddy.replace('Rainer', 'Rainier')
        
    except:
        addyErr = True   # Identify error producing address
        
        return addyErr, addy
    
    return addyErr, cleanAddy


In [5]:
#Split prepopulated address field to obtain only the street address field, i.e. "Address1"

def get_StreetAddress(addy_str):
    
    try:
    
      streetAddy = addy_str.split(',')[0]
        
    except:
        
        streetAddy = '' # Return a blank street address, if error
    
    return streetAddy
    

In [6]:
# Map business address to LocationIQ parameters
# Returns dict for use in LocationIQ API

def get_SplitAddy (splitAddy_keyMap, addyRow):
    
    splitAddy_keys = list(splitAddy_keyMap.keys())
    splitAddy_idx = list(splitAddy_keyMap.values())
    splitAddy_vals = []
    
    for idxval in splitAddy_idx:
        splitAddy_vals.append(addyRow[idxval])
        
    splitAddy_dict = dict(zip(splitAddy_keys, splitAddy_vals))
    
    return splitAddy_dict
        
    
    

In [7]:
# Calls LocationIq API with mapped address parameters
# Reads returned JSON file to find lat/long
# Returns: lat, long, response object, and error dict

def get_LatLong (params_dict, count):
    
    error_dict = {}
    
    locIqURL = 'https://us1.locationiq.com/v1/search.php'
    
    #LocIqToken ='myToken'
    
    #params = {'key': LocIqToken, 'q': addy_string, 'format':'json'}
    
    resp = requests.get(locIqURL, params_dict)
    
    try: 
        locIqResponse = resp.json()
    
        r_lat = float(resp.json()[0]['lat'])

        r_long = float(resp.json()[0]['lon'])
        
    except Exception as e:
        
        r_lat = None
        r_long = None
        
        error_dict = {str(count):e}
    
    return r_lat, r_long, locIqResponse, error_dict

In [8]:
# Function to remove rows with Nan values in any row of a passes list of col names
# Intended for use with removing rows missing lat/long

def removeNanRows(df, checkCol_ls):
    
    for col in checkCol_ls:
        
        df.dropna(axis = 0, subset = [col], inplace = True)
        
    df.reset_index(drop = True, inplace = True)

    
    return df          

##### Begin main section 
The main section of the program follows these steps. 
- Load and transform source data 
- Clean and transform addresses 
- Get lat/long, using LocationIq API
- Remove rows with missing data
- Write out to Excel, for use with ArcGIS Pro

In [9]:
# Read file of FJ locations. This client supplied file contains a multi-level index.
# Decision to select a single-level index to simplify dtype assignment 

# Option 1: For selecting the mulit-level column index 

#rBeachFJ_df = pd.read_excel(rBeachFJ_path, sheet_name = 'Rainier Beach Food', header = [0,1],  dtype = rBeachFJ_dtypes)


# Option 2: For selecting only row 0 as column index; simplifies dtype assignment 
# Skips row 1 to avoid creating a multi-level index

rBeachFJ_df = pd.read_excel(rBeachFJ_path, sheet_name = 'Rainier Beach Food', header = [0],  dtype = rBeachFJ_dtypes,
                            skiprows = lambda x: x in [1])

# Drop unneeded column
rBeachFJ_df.drop(columns = ['Commercial kitchen available to businesses? '], inplace = True)

# Assign new column names
rBeachFJ_df.columns = rBeachFJ_newCols

In [10]:
# Fill map categories with Yes or No values; replace NaN and 'x'

for col in mapCatCols:
    
    for i in range(len(rBeachFJ_df)):
        
        if rBeachFJ_df.loc[i, col] == 'x':
            
            rBeachFJ_df.loc[i, col] = 'Yes'
            
        else:
        
             rBeachFJ_df.loc[i, col] = 'No'

In [11]:
# Assign new col index orders

rBeachGis_newIndex = ['LocID', 'FoodBusiness_YN', 'Business_Name', 'Business_Address', 'State','Zip','Latitude', 'Longitude',
                       'Website', 'Category', 'Food_Swamp', 'Food_Desert','BIPOC_Owned', 'Free_Food', 'Farms', 'Land_Ownership','Commercial_Kitchens', ]

rBeachFJ_df = rBeachFJ_df.reindex(rBeachGis_newIndex, axis = 'columns')

In [12]:
# Iterate through df to clean addresses and split street address 

#rBeachFJ_df['Business_Address'] = rBeachFJ_df.Business_Address.apply(lambda x : str(x).replace('\n', ', '))

badAddyIdx_ls =[]

for i in range(len(rBeachFJ_df)):
    
    cleanAddyErr, rBeachFJ_df.loc[i, 'Business_Address']  = clean_address( rBeachFJ_df.loc[i, 'Business_Address'])\
    
    if cleanAddyErr:
        badAddyIdx_ls.append(i) # Capture index values of addresses that produce errors
        
    rBeachFJ_df.loc[i,'StreetAddress'] = get_StreetAddress(rBeachFJ_df.loc[i, 'Business_Address'])

# Assign Seattle as the city name for all locations. 

rBeachFJ_df['City'] = 'Seattle'

In [13]:
# Use function to get lat/long from LocationIQ

startTime = time.time() # Start timer, to learn how long geocoding takes to complete 

# Create empty lists for capturing API request and return information 
addyParams_ls = [] # For API request dicts
addyResponse_ls = [] # For API response 
addyError_ls = [] #For debug

for i in range(len(rBeachFJ_df)):
    
    #Select row from df and get split addy
    nextRow = rBeachFJ_df.iloc[i]    
    nextAddy_dict = get_SplitAddy(splitAddy_keyMap, nextRow)

    # Combine address with standard API request info, store in list for future reference
    nextAddyParams = {**locIqParams, **nextAddy_dict}
    addyParams_ls.append(nextAddyParams)
    
    # Call LocationIq API via function 
    tempLat, tempLong, addyResponse, addyError = get_LatLong(nextAddyParams, i)

    # Append lat/long info to the FJ location 
    rBeachFJ_df.loc[i, 'Latitude'] = tempLat
    rBeachFJ_df.loc[i, 'Longitude'] = tempLong
    
    # Store API response and any error 
    addyResponse_ls.append(addyResponse)
    addyError_ls.append(addyError)
    
    # Pause for loop to avoid API throttling 
    time.sleep(1.1)


endTime = time.time()

# Calcuate elapsed time
elapsedTime = endTime - startTime
print('Elapsed time: ', elapsedTime)


Elapsed time:  499.1313512325287


In [14]:
# Remove any locations missing lat/long (can't be mapped) and reset index. 

allFjLoc_df = removeNanRows(rBeachFJ_df, ['Latitude', 'Longitude'])
allFjLoc_df.reset_index(drop = True, inplace = True)

In [15]:
# Split out the desired categories, because ArcPro GIS "view filters" don't work on the web map  

bipoc_df = allFjLoc_df[allFjLoc_df.BIPOC_Owned == 'Yes']
foodSwamp_df = allFjLoc_df[allFjLoc_df.Food_Swamp == 'Yes']
foodDesert_df = allFjLoc_df[allFjLoc_df.Food_Desert == 'Yes']
freeFood_df = allFjLoc_df[allFjLoc_df.Free_Food == 'Yes']
farms_df = allFjLoc_df[allFjLoc_df.Farms == 'Yes']


In [16]:
# Write out FJ locations, by categories, for use by ArcPro GIS 

with pd.ExcelWriter(r'C:\\Users',# path to output file 
                    date_format='%Y-%m-%d-%H-%M-%S', datetime_format='YYYY-MM-DD-HH:MM:SS') as mapData_writer:
    
    rBeachFJ_df.to_excel(mapData_writer, sheet_name = 'All FJ Locations', index = False)
    allFjLoc_df.to_excel(mapData_writer, sheet_name = 'API Found Locations', index = False)
    bipoc_df.to_excel(mapData_writer, sheet_name = 'BIPOC Owned Locations', index = False)
    foodSwamp_df.to_excel(mapData_writer, sheet_name = 'Food Swamp Locations', index = False)
    foodDesert_df.to_excel(mapData_writer, sheet_name = 'Food Desert Locations', index = False)
    freeFood_df.to_excel(mapData_writer, sheet_name = 'Free Food Locations', index = False)
    farms_df.to_excel(mapData_writer, sheet_name = 'Farm Locations', index = False)